### Importamos las bibliotecas que vamos a utilizar.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from googletrans import Translator

### Obtenemos los dispositivos a buscar.

Cargamos el dataset de DXO Mark y guardamos todos los dispostivos en una lista para buscar comentarios en internet sobre ellos.

In [53]:
dxo = pd.read_csv("dxomark.csv", sep=";", index_col=False)
s = list(dxo["Device"]) #creamos una lista con los smartphones que tenemos en dxomark

Iniciamos un webdriver en selenium.

In [65]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=option) #iniciamos nuestro webdriver

### Buscamos los datos

Probamos con el 4 disitivo de nuestra y vamos buscando donde interactuar con la web hasta llegar a los comentarios deseados.
Google ofrece con su herramienta Shooping comentarios sobre los productos que obtiene de la mayoria de tiendas online (Amazon, fnac, páginas de los fabricantes, etc.). Nuestro objetivo es para los dispositivos que tengan disponibles esos comentarios, interacturar con la página hasta llegar con ellos y obtenerlos.

In [88]:
p = s[4].replace(" ", "+").replace("(","").replace(")","").replace("5G", "").replace(" Snapdragon","").replace(" Exynos", "")
url = f"https://www.google.com/search?q={p}"
driver.get(url)
detector = Translator()

try:
    clear_button = driver.find_element(By.XPATH,'//button[@id="L2AGLb"]').click() #Pulsamos el boton de las cookies
except:
    pass

Si existe el boton reseñas, es que existen y le damos click, en caso contrario lo imprime por consola que no tiene comentarios.


In [85]:
try:
    clear_button = driver.find_element(By.XPATH,'//button[@title="Reseñas"]').click()
except:
    print("No tiene comentarios")


Damos click al boton de más comentarios hasta que obtengamos todos los comentarios existenes.

In [80]:
while True:
    try:
        clear_button = driver.find_element(By.XPATH,'//button[@jsname="LrJMvf"]').click()
        time.sleep(0.1)
    except:
        try:
            time.sleep(0.3)
            clear_button = driver.find_element(By.XPATH,'//button[@jsname="LrJMvf"]').click()
        except:
            print("Ya estan todos los comentarios")
            break


Ya estan todos los comentarios


Expandimos los comentarios largos.


In [87]:
while True:
    try:
        clear_button = driver.find_element(By.XPATH,'//button[@jsaction="trigger.M7VP"][@aria-expanded="false"]').click()
        time.sleep(0.1)
    except:
        try:
            time.sleep(1)
            clear_button = driver.find_element(By.XPATH,'//button[@jsaction="trigger.M7VP"][@aria-expanded="false"]').click()
        except:
            print("Ya estan extendidos todos los comentarios")
            break

Ya estan extendidos todos los comentarios


Transformamos la web en un objeto tipo Soup y obtenemos toda la información relevante de los comentarios.

In [58]:
def val_a_int(text):
    text = text.replace("Valoración de ", "").replace(",0 de un máximo de 5,", "")
    return(int(text))
    
soup = BeautifulSoup(driver.page_source,"lxml")
soup = soup.find_all("div", {"jsname":"GYYgbc"})
dic_res={"device":[], "titulo":[],"valoracion":[],"comentario":[]}
for com in soup:
    for comentario in com.find_all("div", {"jsname":"UtCV2e"}):
        if comentario.find("span", {"class":"KTsGge"}) != None:    
            dic_res["titulo"].append(comentario.find("span", {"class":"KTsGge"}).text)
        else:
            dic_res["titulo"].append("Sin titulo")
        dic_res["valoracion"].append(val_a_int(comentario.find("span", {"class":"Fam1ne tPhRLe"}).get("aria-label")))
        dic_res["comentario"].append(comentario.find("div", {"jsname":"Lvg1qb"}).text)
        dic_res["device"].append(s[32])

df = pd.DataFrame(dic_res)
df.head(10)

,device,titulo,valoracion,comentario
0,Vivo X60 Pro 5G (Snapdragon),Best camera centric phone with excellent in ha...,5,Best camera centric phone in this price range....
1,Vivo X60 Pro 5G (Snapdragon),Surprisingly it's Vivo smartphone.,5,Beautiful Design & build.\nAwesome rear camera...
2,Vivo X60 Pro 5G (Snapdragon),Sin titulo,5,Best mobile. I purchased in 31000.00 with disc...
3,Vivo X60 Pro 5G (Snapdragon),Got it for 30k,5,"Superb. Camera and display. Zoom, potraits"
4,Vivo X60 Pro 5G (Snapdragon),very happy with this product,5,all round good very good camera fantastic
5,Vivo X60 Pro 5G (Snapdragon),Sin titulo,5,बहोत बढ़िया फोन सबकुछ अच्छा है सुंदर डिज़ाइन के ...
6,Vivo X60 Pro 5G (Snapdragon),Sin titulo,4,"So far, no issues; I like the camera, comfort ..."


In [81]:
df.to_csv("reviews (small sample).csv", sep=";")

### Recolección de todos los datos.
Una vez visto que nuestro código funciona, lo juntamos todo y realizamos una busqueda para todos los dispositivos en nuestro dataset.

In [16]:
def val_a_int(text):
    text = text.replace("Valoración de ", "").replace(",0 de un máximo de 5,", "")
    return(int(text))

dxo = pd.read_csv("dxomark.csv", sep=";", index_col=False)
s = list(dxo["Device"])

#creamos una lista con los smartphones que tenemos en dxomark
dic_res={"manufacturer":[],"device":[],"valoracion":[],"comentario":[]}
option = webdriver.ChromeOptions()
#option.add_argument('headless') #Si no se quiere ver el driver
driver = webdriver.Chrome(options=option) #iniciamos nuestro webdriver


for product in s:
    p = product.replace(" ", "+").replace("(","").replace(")","").replace("5G", "").replace("Snapdragon","").replace("Exynos", "")
    url = f"https://www.google.com/search?q={p}&biw=1504&bih=884"
    driver.get(url)
    try:
        #Pulsamos el boton de las cookies
        clear_button = driver.find_element(By.XPATH,'//button[@id="L2AGLb"]').click()
    except:
        pass
    #Si existe el boton reseñas, es que existen, en caso contrario lo imprime por consola
    try:
        clear_button = driver.find_element(By.XPATH,'//button[@title="Reseñas"]').click()
    except:
        continue #no tiene comentarios
    #Extendemos para obtener todos los comentarios existentes.
    i = 300 #limitamos el nuemro de veces que se puede dar a más comentarios
    while True:
        try:
            clear_button = driver.find_element(By.XPATH,'//button[@jsname="LrJMvf"]').click()
            i = i -1
            if i < 0: 
                break
            time.sleep(0.1)
        except:
            try:
                time.sleep(1.5)
                clear_button = driver.find_element(By.XPATH,'//button[@jsname="LrJMvf"]').click()
                i = i -1
                if i < 0: 
                    break
            except:
                print("Ya estan todos los comentarios")
                break

    soup = BeautifulSoup(driver.page_source,"lxml")
    soup = soup.find_all("div", {"jsname":"GYYgbc"})
    for com in soup:
        for comentario in com.find_all("div", {"jsname":"UtCV2e"}):
            if comentario.find("span", {"class":"KTsGge"}) != None:
                dic_res["comentario"].append(comentario.find("span", {"class":"KTsGge"}).text + "\n" + comentario.find("div", {"jsname":"Lvg1qb"}).text)
            else:
                dic_res["comentario"].append(comentario.find("div", {"jsname":"Lvg1qb"}).text)

            dic_res["valoracion"].append(val_a_int(comentario.find("span", {"class":"Fam1ne tPhRLe"}).get("aria-label")))
            dic_res["device"].append(product)
            dic_res["manufacturer"].append(product.split()[0])
    print(product, len(dic_res["comentario"]))

df = pd.DataFrame(dic_res)
df.to_csv("original_reviews.csv", sep=";")

Samsung Galaxy S22 Ultra (Exynos) 1840
Samsung Galaxy S21 FE 5G (Snapdragon) 3040
Ya estan todos los comentarios
Realme GT Neo 2 5G 3078
Ya estan todos los comentarios
Vivo V21 5G 3097
Ya estan todos los comentarios
Oppo A16s 5G 3129
Google Pixel 6 5459
Ya estan todos los comentarios
Sony Xperia 1 III 5656
Ya estan todos los comentarios
Vivo X70 Pro+ 5663
Samsung Galaxy Z Fold3 5G 7323
Ya estan todos los comentarios
Motorola Edge 20 Pro 7391
Ya estan todos los comentarios
Oppo Reno6 Pro 5G 7510
Ya estan todos los comentarios
Oppo Reno6 5G 7530
Ya estan todos los comentarios
Xiaomi 11T 7900
Google Pixel 6 Pro 10280
Ya estan todos los comentarios
Xiaomi 11T Pro 10583
Ya estan todos los comentarios
Oppo Reno6 Pro+ 10702
Ya estan todos los comentarios
OnePlus Nord CE 5G 10968
Apple iPhone 13 Pro Max 13388
Ya estan todos los comentarios
Apple iPhone 13 mini 14793
Ya estan todos los comentarios
Xiaomi Mi 11 Pro 14795
Ya estan todos los comentarios
Vivo Y72 5G 14852
Ya estan todos los comenta

Traducimos los comentarios al ingles y los guardamos.

In [ ]:
df_long = pd.read_csv("original_reviews.csv", sep=";")

def trasnlate(text):
    try:
        detector = Translator()
        dec_lan = detector.translate(text)
    except:
        try:
            time.sleep(4)
            detector = Translator()
            dec_lan = detector.translate(text)
        except:
            return text
        
    return dec_lan.text


df_long["comentario"] = df_long["comentario"].map(lambda x: trasnlate(x))
df_long

In [10]:
df_long = df_long[["manufacturer","device","valoracion","comentario"]]
df_long.to_csv("english_reviews.csv", sep=";", index=False)


In [11]:
df_long.head()

,manufacturer,device,valoracion,comentario
0,Samsung,Samsung Galaxy S22 Ultra (Exynos),5,"Incredible screen, cameras, battery, pen........."
1,Samsung,Samsung Galaxy S22 Ultra (Exynos),1,Watch out for Samsung. Eye replacement policie...
2,Samsung,Samsung Galaxy S22 Ultra (Exynos),4,"Very good, but the software needs optimization..."
3,Samsung,Samsung Galaxy S22 Ultra (Exynos),5,Impressive\nIt is a renewed mobile with respec...
4,Samsung,Samsung Galaxy S22 Ultra (Exynos),5,An aesthetic and functional jewel.\nAfter my N...
